# config

> Utilities for loading Stata and nbstata

Before we can use [pystata](https://www.stata.com/python/pystata/index.html), we need to find the local Stata path (i.e., `find_path`) and then [add pystata to sys.path](https://www.stata.com/python/pystata/install.html#method-2-adding-pystata-to-sys-path) (i.e., `set_pystata_path`) so it can be imported.

The `get_config` function handles nbstata configuration, more broadly.

In [ ]:
#| default_exp config
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

## pystata configuration

In [ ]:
#| export
import os
import sys
import platform
from shutil import which
from pathlib import Path
from packaging import version
from configparser import ConfigParser, NoSectionError

In [ ]:
#| export
def win_find_path():
    import winreg
    reg = winreg.ConnectRegistry(None, winreg.HKEY_CLASSES_ROOT)
    subkeys = [r'Stata17Do\shell\do\command',
                r'Stata18Do\shell\do\command',
                r'Stata19Do\shell\do\command']

    fpath = ''
    for subkey in subkeys:
        try:
            key = winreg.OpenKey(reg, subkey)
            fpath = winreg.QueryValue(key, None).split('"')[1]
        except FileNotFoundError:
            pass
        if fpath:
            break

    return fpath


def mac_find_path():
    """
    Attempt to find Stata path on macOS when not on user's PATH.
    Modified from stata_kernel's original to only location "Applications/Stata". 

    Returns:
        (str): Path to Stata. Empty string if not found.
    """
    path = Path('/Applications/Stata')
    if not path.exists():
        return ''
    else:
        return str(path)

In [ ]:
#| export
def find_path():
    if os.getenv('CONTINUOUS_INTEGRATION'):
        print('WARNING: Running as CI; Stata path not set correctly')
        return 'stata'
    if platform.system() == 'Windows':
        return win_find_path()
    elif platform.system() == 'Darwin':
        return mac_find_path()
    else:
        for i in ['stata-mp', 'stata-se', 'stata']:
            stata_path = which(i)
            if stata_path:
                break
        return stata_path

In [ ]:
find_path()

'C:\\Program Files\\Stata17\\StataMP-64.exe'

In [ ]:
#| export
def find_dir_edition():
    stata_path = find_path()
    if stata_path is None:
        raise OSError("Stata path not found.")
    stata_dir = str(os.path.dirname(stata_path))
    stata_exe = str(os.path.basename(stata_path)).lower()

    edition = 'be'
    for e in ('se','mp'):
        if stata_exe.find(e) > -1:
            edition = e            
    return (stata_dir,edition)

In [ ]:
#| export
def set_pystata_path(path=None):
    if path == None:
        path, _ = find_dir_edition()
    if not os.path.isdir(path):
        raise OSError(path + ' is invalid')
    if not os.path.isdir(os.path.join(path, 'utilities')):
        raise OSError(path + " is not Stata's installation path")
    sys.path.append(os.path.join(path, 'utilities'))

In [ ]:
#| export
def launch_stata(path=None, edition=None, splash=True):
    """
    We modify stata_setup to make splash screen optional
    """
    if path == None or edition == None:
        path_found, edition_found = find_dir_edition()
        path = path_found if path==None else path
        edition = edition_found if edition==None else edition
    set_pystata_path(path)
    import pystata
    if version.parse(pystata.__version__) >= version.parse("0.1.1"):
        # Splash message control is a new feature of pystata-0.1.1
        pystata.config.init(edition,splash=splash)
    else:
        pystata.config.init(edition)

In [ ]:
#|eval: false
launch_stata(splash=False)
import pystata
pystata.config.status()

    System information
      Python version         3.10.4
      Stata version          Stata 17.0 (MP)
      Stata library path     C:\Program Files\Stata17\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = default, height = default
      graphic format         svg


## nbstata configuration

In [ ]:
#| export
def get_config():
    """First check if a configuration file exists. If not, try `find_dir_edition`."""
    global_config_path = Path(os.path.join(sys.prefix,'etc','nbstata.conf'))
    user_config_path = Path('~/.nbstata.conf').expanduser()
    env = {'stata_dir': None,
           'edition': None,
           'graph_format': 'png',
           'echo': 'None',
           'splash': 'False',
           'missing': '.',
          }
    for cpath in (global_config_path, user_config_path):
        try:
            if cpath.is_file():
                config = ConfigParser()
                config.read(str(cpath))
                env.update(dict(config.items('nbstata')))
        except:
            pass
    if env['stata_dir']==None or env['edition']==None:     
        stata_dir,stata_ed = find_dir_edition()     
        default_env = {'stata_dir': stata_dir, 'edition': stata_ed}
        env.update(default_env)
    if env['echo'] not in {'True', 'False', 'None'}:
        env['echo'] = 'None'
    return env

In [ ]:
#|eval: false
get_config()

{'stata_dir': 'C:\\Program Files\\Stata17',
 'edition': 'mp',
 'graph_format': 'png',
 'echo': 'None',
 'splash': 'False',
 'missing': '.'}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()